In [11]:
import os

# Check current working directory
print(os.getcwd())

# Change to the root directory of your project, if necessary
os.chdir('C:/Users/yashv/Downloads/HADLN_implementation/HADLN_implementation/')
print(os.getcwd())


C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation
C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from datetime import datetime
from preprocessing.lstm_preprocessing import load_data, split_data  # Update this import path as needed
from models.cnn_bilstm import build_model  # Import the LSTM model

In [14]:
csv_file_path = 'data/raw/physionet2017.csv'  # Update the path to your CSV file
model_path = 'models/cnnbilstm_best.h5'
log_dir = f"results/tensorboard/CNNBiLSTM/{datetime.now().strftime('%Y%m%d-%H%M%S')}"

# Create directories if they don't exist
os.makedirs(log_dir, exist_ok=True)
os.makedirs('models', exist_ok=True)

# Load and prepare data
ecg_signals, labels_one_hot, num_classes = load_data(csv_file_path)

# Convert one-hot labels back to integer labels for class weights calculation
labels_integer = np.argmax(labels_one_hot, axis=1)

# Split data into training and testing sets
X_train, X_test, y_train_one_hot, y_test_one_hot = split_data(ecg_signals, labels_one_hot)



In [15]:
# Extract time_step and num_features
time_step = X_train.shape[1]
num_features = X_train.shape[2]


# Build and compile the LSTM model
model = build_model(time_step, num_features, num_classes)

# Define a learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.5,
                                 patience=3,
                                 min_lr=1e-8)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # Categorical crossentropy for one-hot labels
              metrics=['accuracy'])



In [17]:
class_weights = {0: 1.0, 1: 6.0, 2: 3.0, 3: 20.0}
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath=model_path, save_best_only=True, verbose=1)
callbacks = [checkpoint_callback, tensorboard_callback, lr_scheduler]

# Train the model with class weights and callbacks
model.fit(X_train, y_train_one_hot,
          validation_data=(X_test, y_test_one_hot),
          epochs=100,
          batch_size=8,
          callbacks=callbacks)

Epoch 1/100
427/427 [==============================] - ETA: 0s - loss: 0.9252 - accuracy: 0.6045
Epoch 1: val_loss improved from inf to 1.22285, saving model to models\cnnbilstm_best.h5
427/427 [==============================] - 11s 25ms/step - loss: 0.9252 - accuracy: 0.6045 - val_loss: 1.2229 - val_accuracy: 0.5950 - lr: 0.0010
Epoch 2/100
425/427 [============================>.] - ETA: 0s - loss: 0.8896 - accuracy: 0.6234
Epoch 2: val_loss improved from 1.22285 to 1.10661, saving model to models\cnnbilstm_best.h5
427/427 [==============================] - 10s 24ms/step - loss: 0.8893 - accuracy: 0.6233 - val_loss: 1.1066 - val_accuracy: 0.5955 - lr: 0.0010
Epoch 3/100
425/427 [============================>.] - ETA: 0s - loss: 0.8586 - accuracy: 0.6363
Epoch 3: val_loss improved from 1.10661 to 1.04382, saving model to models\cnnbilstm_best.h5
427/427 [==============================] - 10s 24ms/step - loss: 0.8591 - accuracy: 0.6362 - val_loss: 1.0438 - val_accuracy: 0.6073 - lr: 0.0

In [ ]:
# Load the trained model
model_path = 'models/cnnbilstm_best.h5'  # Adjust path if necessary
model = tf.keras.models.load_model(model_path)
